In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from numpy.random import RandomState
rdm = RandomState(1)

y_  = tf.constant(rdm.rand(1,10))
y = np.array([0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0])

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_)
with tf.Session() as sess:
    loss = sess.run(cross_entropy)
    print(loss)

y_softmax = tf.nn.softmax(logits=y_)
cross_entropy_v1 = -tf.reduce_sum(y * tf.log(y_softmax))

with tf.Session() as sess:
    loss = sess.run(cross_entropy_v1)
    print(loss)

[1.91865955]
1.9186595478618151


In [3]:
logits = tf.constant([[1.0,2.0,3.0],[1.0,2.0,3.0],[1.0,2.0,3.0]])
y  = tf.nn.softmax(logits)
y_ = tf.constant([[0.0,0.0,1.0],[0.0,0.0,1.0],[0.0,0.0,1.0]])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

cross_entropy2 = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

with tf.Session() as sess:
    c_e = sess.run(cross_entropy)
    c_e2 = sess.run(cross_entropy2)
    print("step2:cross_entropy result=")
    print(c_e)
    print("Function(softmax_cross_entropy_with_logits) result=")
    print(c_e2)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step2:cross_entropy result=
1.222818
Function(softmax_cross_entropy_with_logits) result=
1.2228179


In [4]:
Truth = np.array([0,0,1,0])
Pred_logits = np.array([3.5,2.1,7.89,4.4])

loss = tf.nn.softmax_cross_entropy_with_logits(labels=Truth,logits=Pred_logits)
loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Truth,logits=Pred_logits)
loss3 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(Truth),logits=Pred_logits)

with tf.Session() as sess:
    print(sess.run(loss))
    print(sess.run(loss2))
    print(sess.run(loss3))

0.04493472585997595
0.04493472585997595
0.04493472585997595


In [5]:
if False:
    batch_size = n
    x = tf.placeholder(tf.float32, shape=(batch_size,2), name='x-input')
    y_ = tf.placeholder(tf.float32, shape=(batch_size,1), name='y-input')
    
    loss = ...
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    with tf.Session() as sess:
        for i in range(STEP):
            current_x, current_y = ...
            sess.run(train_step, feed_dict={
                x : current_x,
                y_ : current_y
            })

**`tf.train.exponential_decay`**

In [ ]:
if False:
    global_step = tf.Variable(0)
    
    # 通过 exponential_decay 函数来生成学习率
    leanring_rate = tf.train.exponential_decay(
        0.1, global_step, decay_steps=100, decay_rate=0.96, staircase=True
    )
    
    # 使用'指数衰减学习率'
    # 在 minimize 函数中传入 global_step, 并将自动更新 global_step 参数
    # 从而使得学习率也得到相应更新
    optimizer = tf.train.GradientDescentOptimizer(leanring_rate)
    learning_step = optimizer.minimize(...my_loss.., global_step=global_step)

**`Regularization`**

In [6]:
weight = tf.constant([[1.0, -2.0], [-3.0, 4.0]])

with tf.Session() as sess:
    l1 = tf.contrib.layers.l1_regularizer(.5)(weight)
    l2 = tf.contrib.layers.l2_regularizer(.5)(weight)
    l1_l2 = tf.contrib.layers.l1_l2_regularizer(.5, .5)(weight)
    
    print(sess.run(l1))
    print(sess.run(l2))
    print(sess.run(l1_l2))

5.0
7.5
12.5


**`Exponential Moving Average`**

In [7]:
var = tf.Variable(0, dtype=tf.float32)

# 模拟社交网络迭代轮数, 动态控制衰减率
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均累
ema = tf.train.ExponentialMovingAverage(decay=0.99, num_updates=step)

# 定义一个滑动平均的操作
# 这里需要给定一个列表, 每次执行这个操作时, 列表里的元素都会被更新
maintain_average_op = ema.apply([var])

In [12]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print(sess.run([var, ema.average(var)]))

    # 更新v1的滑动平均值，衰减率为min{0.99,(1+step)/(10+step)=0.1}=0.1,
    # 所以v1的滑动平均被更新为0.1*0+0.9*5=4.5
    sess.run(tf.assign(var, 5))
    sess.run(maintain_average_op)
    print(sess.run([var, ema.average(var)]))
    
    # decay = min{0.99, (1+step)/(10+step)} = 0.99
    # 0.99 * 4.5 + 0.01 * 10 = 4.555
    sess.run(tf.assign(step, 10000))
    sess.run(tf.assign(var, 10))
    sess.run(maintain_average_op)
    print(sess.run([var, ema.average(var)]))
    
    # 0.99 * 4.555 + 0.01 * 10
    sess.run(maintain_average_op)
    print(sess.run([var, ema.average(var)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
